# Compile graql schema from graql data source

## setup

### constants

In [18]:
dir_out = "/Users/rkm916/Sync/projects/2019-02-knowledgebase/src/src_gql/"
filename_out = "schema_upper_bfo_compiled.gql"

### modules

In [19]:
from grakn.client import GraknClient
import datetime

### In terminal

#### start grakn

grakn server start

#### load the meta schema

grakn console --keyspace meta --file /Users/rkm916/Sync/projects/2019-02-knowledgebase/src/src_gql/schema_meta.gql 

#### load the schema to be parsed from instance 'meta' form to class form

grakn console --keyspace meta --file /Users/rkm916/Sync/projects/2019-02-knowledgebase/src/src_gql/schema_upper_BFO_meta.gql

In [42]:
list_define_block = [] # this is the list of statements that will define classes in the schema

### open client to grakn server

In [43]:
# NB: we need to remember to close the client when done
client = GraknClient(uri="localhost:48555")

### load meta keyspace

In [45]:
client.keyspaces().retrieve()

['meta', 'grakn', 'bfo']

In [52]:
sessionMeta = client.session(keyspace="meta") 

### add entity, relation and attribute classes

In [53]:
# https://dev.grakn.ai/docs/client-api/python#api-reference

    ## session is open
with sessionMeta.transaction().read() as read_transaction:
    
    for match_type in ("type-entity", "type-relation", "type-attribute"):
        query_things = "match $x isa {0}, has name $n; get $n; limit 9999;".format(match_type) #https://docs.python.org/3/library/string.html

        infer = True

        answer_iterator_things = read_transaction.query(query=query_things, infer=infer) # returns an iterater of answer

        answers_things = answer_iterator_things.collect_concepts() # convert the iterator into a list of Concepts - in this case, Things

        # iterate over things, each corresponding to a class we want to define

        for answer in answers_things:

            name = answer.value()

            if match_type == "type-relation":

                # retrieve the roles that this relation relates

                query_roles_related = "match $rp (playing-role: $pr, is-relation: $ir) isa roleplaying; $ir has name '{!s}'; $rp has name $n; get $n;".format(name)

                #print(query_roles)

                answer_iterator_roles_related = read_transaction.query(query=query_roles_related, infer=infer) # returns an iterator of type-relation entity-type things

                answers_roles_related = answer_iterator_roles_related.collect_concepts() # convert the iterator into a list of Concepts - in this case, Things

                roles_related = [answer.value() for answer in answers_roles_related]

            # retrieve the roleplaying relations where this thing plays the role 'plays-role'

            query_roles_played = "match $rp (playing-role: $pr, is-relation: $ir) isa roleplaying; $pr has name '{!s}'; $rp has name $n; get $n;".format(name)

            #print(query_roles)

            answer_iterator_roles_played = read_transaction.query(query=query_roles_played, infer=infer) # returns an iterator of type-relation entity-type things

            answers_roles_played = answer_iterator_roles_played.collect_concepts() # convert the iterator into a list of Concepts - in this case, Things

            roles_played = [answer.value() for answer in answers_roles_played]

            # print(roles)

            if match_type in ("type-entity", "type-relation"):
                # in a similar manner retrieve the attributes

                query_attrs = "match $havingattr (has-attribute: $has-a, is-attribute-of: $iao) isa having-attribute; $has-a has name '{!s}'; $havingattr has name $n; get $n;".format(name)

                #print(query_attrs)

                answer_iterator_attrs = read_transaction.query(query=query_attrs, infer=infer) # returns an iterator of type-attribute entity-type things

                answers_attrs = answer_iterator_attrs.collect_concepts() # convert the iterator into a list of Concepts - in this case, Things

                attributes = [answer.value() for answer in answers_attrs]

                # print(attributes) # we expect this to be empty for bfo..

            elif match_type == "type-attribute":

                # get the attr-datatype value

                query_datatype = "match $attr isa attribute, has name '{!s}', has attr-datatype $dt;".format(name)

                answer_iterator_datatype = read_transaction.query(query=query_datatype, infer=infer) # returns an iterator of type-attribute entity-type things

                answer_datatype = nswer_iterator_datatype.collect_concepts()

                datatype = answers_datatype[0].value()

            # in a similar manner find out if the class has any parents

            query_parentship = "match $ps (parent: $p, child: $c) isa parentship; $c has name '{!s}'; $p has name $n; get $n;".format(name)

            answer_iterator_parentship = read_transaction.query(query=query_parentship, infer=infer)

            answers_parentship = answer_iterator_parentship.collect_concepts() # convert the iterator into a list of Concepts - in this case, Things

            if len(answers_parentship) == 1:
                parent = answers_parentship[0].value()
            elif len(answers_parentship) == 0:
                parent = match_type.split("-")[1]

            # print(parent) # we expect this to be empty for bfo..

            # Finally retrieve abstract status

            query_isabstr =  "match $x isa {0}, has name '{1}', has isabstract $ia; get $ia;".format(match_type,name)


            answer_iterator_isabstr = read_transaction.query(query=query_isabstr, infer=infer)

            answers_isabstr = answer_iterator_isabstr.collect_concepts() # convert the iterator into a list of Concepts - in this case, Things

            isabstract = answers_isabstr[0].value()

            # print(isabstract)

            # Write to the schema file

            define_block = '\n{0} sub {1}'.format(name, parent)

            if match_type == "type-relation":
                for role in roles_related:
                    define_block = "{0},\n    relates {1}".format(define_block, role)

            if match_type == "type-attribute":
                define_block = "{0},\n    datatype {1}".format(define_block, datatype)

            for role in roles_played:
                define_block = "{0},\n    plays {1}".format(define_block, role)

            for attributes in attributes:
                define_block = "{0},\n    has {1}".format(define_block, attribute)

            if isabstract:
                define_block = "{0},\n    abstract".format(define_block)

            define_block = "{0};\n".format(define_block)

            list_define_block.append(define_block)

Iterate over rules:

In [48]:
# https://dev.grakn.ai/docs/client-api/python#api-reference
# TODO


## wrap up

In [54]:
sessionMeta.close()

In [55]:
client.keyspaces().delete("meta") # wipe the keyspace i.e. instances of the ontology

In [56]:
client.close()

## write out new schema to .gql file

In [57]:
with open("{0}{1}".format(dir_out, filename_out), 'w') as f:
    f.write("# compiled {0}\n".format(datetime.datetime.now()))
    f.write("\n")
    f.write("define\n")
    for define_block in list_define_block:
        f.write(define_block)